### Exam-3  by Aniket Amar Thopte

###### The exam problem statement deals with hourly probabilistic load forecasting. We have been given dataset from 2002 to 2006 (i.e. 5 years data) on hourly basis. 

####  Import all necessary libraries

In [311]:
import h5py
import datetime
import calendar
import warnings
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

### 1- Read the data file

In [312]:
df = pd.read_excel("history.xlsx")
df = df.dropna(axis = 0) # Remove rows with NaN data
df = df.reset_index(drop = True) # Replace the previous index with new index
df["DATE"] = pd.to_datetime(df["DATE"], format = "%Y-%m-%d") # Specify the format of entry date
df["Year"] = df["DATE"].dt.year
df["Month"] = df["DATE"].dt.month
df["Day"] = df["DATE"].dt.day
df["Weekday"] = df["DATE"].dt.weekday + 1 # To adjust to 1 to 7 instead of 0 to 6
df["Week"] = df["Day"].apply(lambda x: (x - 1) // 7 + 1)
df["Weekend"] = df["Weekday"].apply(lambda x: 1 if x >= 6 else 0) # Apply function to the column (default: axis=0)
df

,DATE,Hour,Temperature,Load,Year,Month,Day,Weekday,Week,Weekend
0,2002-01-01,1,43.72,1384494.0,2002,1,1,2,1,0
1,2002-01-01,2,42.72,1392822.0,2002,1,1,2,1,0
2,2002-01-01,3,41.84,1407887.0,2002,1,1,2,1,0
3,2002-01-01,4,41.04,1438658.0,2002,1,1,2,1,0
4,2002-01-01,5,40.56,1484046.0,2002,1,1,2,1,0
...,...,...,...,...,...,...,...,...,...,...
43819,2006-12-31,20,71.48,1612494.0,2006,12,31,7,5,1
43820,2006-12-31,21,70.88,1473990.0,2006,12,31,7,5,1
43821,2006-12-31,22,70.32,1374181.0,2006,12,31,7,5,1
43822,2006-12-31,23,69.84,1272117.0,2006,12,31,7,5,1


### 2- Data pre-processing

In [313]:
# Find last Monday for holiday
import time
import calendar
import datetime

def last_mon_date(year, month):
  
  """
  Returns a matrix representing a month’s calendar
  Each row represents a week; days outside of the month a represented by zeros
  Each week begins with Monday
  """
  cal = calendar.monthcalendar(year, month)
  last_mon_date = cal[4][0] if (cal[4][0] > 0) else cal[3][0]
  return str(year)+"-"+str(month)+"-"+str(last_mon_date)

unique_year = df["Year"].unique()
last_mon_may = []

for i in range(0, unique_year.shape[0], 1):
  last_mon_may.append(last_mon_date(unique_year[i], 5))
  
# Convert to timestamp
last_mon_may = [time.mktime(datetime.datetime.strptime(x,"%Y-%m-%d").timetuple()) for x in last_mon_may]

In [314]:
# Find holidays - total number of holidays = 5* 10 * 24 (years*days*hours) = 1200
df["Holiday"] = 0
df["Holiday"] = df["DATE"].apply(lambda x: 1 if (datetime.datetime.timestamp(x) in last_mon_may) else 0)
df["Holiday"].loc[(df["Month"] == 1) & (df["Day"] == 1)] = 1 # Remember () for condition
df["Holiday"].loc[(df["Month"] == 12) & (df["Day"] == 25)] = 1
df["Holiday"].loc[(df["Month"] == 11) & (df["Day"] == 11)] = 1
df["Holiday"].loc[(df["Month"] == 7) & (df["Day"] == 4)] = 1
df["Holiday"].loc[(df["Month"] == 1) & (df["Week"] == 3) & (df["Weekday"] == 1)] = 1
df["Holiday"].loc[(df["Month"] == 2) & (df["Week"] == 3) & (df["Weekday"] == 1)] = 1 
df["Holiday"].loc[(df["Month"] == 11) & (df["Week"] == 4) & (df["Weekday"] == 4)] = 1
df["Holiday"].loc[(df["Month"] == 10) & (df["Week"] == 2) & (df["Weekday"] == 1)] = 1
df["Holiday"].loc[(df["Month"] == 9) & (df["Week"] == 1) & (df["Weekday"] == 1)] = 1

print ("Total number for holidays: {} \n".format(np.sum(df["Holiday"])))

Total number for holidays: 1200 



### 3- Data Re-arrangement

In [315]:
df = df[["DATE", "Hour", "Load","Temperature","Year","Month","Day","Weekday","Week","Weekend","Holiday"]]
df

,DATE,Hour,Load,Temperature,Year,Month,Day,Weekday,Week,Weekend,Holiday
0,2002-01-01,1,1384494.0,43.72,2002,1,1,2,1,0,1
1,2002-01-01,2,1392822.0,42.72,2002,1,1,2,1,0,1
2,2002-01-01,3,1407887.0,41.84,2002,1,1,2,1,0,1
3,2002-01-01,4,1438658.0,41.04,2002,1,1,2,1,0,1
4,2002-01-01,5,1484046.0,40.56,2002,1,1,2,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...
43819,2006-12-31,20,1612494.0,71.48,2006,12,31,7,5,1,0
43820,2006-12-31,21,1473990.0,70.88,2006,12,31,7,5,1,0
43821,2006-12-31,22,1374181.0,70.32,2006,12,31,7,5,1,0
43822,2006-12-31,23,1272117.0,69.84,2006,12,31,7,5,1,0


In [316]:
df_backup = df.copy()

df.head()


,DATE,Hour,Load,Temperature,Year,Month,Day,Weekday,Week,Weekend,Holiday
0,2002-01-01,1,1384494.0,43.72,2002,1,1,2,1,0,1
1,2002-01-01,2,1392822.0,42.72,2002,1,1,2,1,0,1
2,2002-01-01,3,1407887.0,41.84,2002,1,1,2,1,0,1
3,2002-01-01,4,1438658.0,41.04,2002,1,1,2,1,0,1
4,2002-01-01,5,1484046.0,40.56,2002,1,1,2,1,0,1


### 4- Model Building

In [317]:
from sklearn.ensemble import GradientBoostingRegressor
N_ESTIMATORS = 100
MAX_DEPTH = 5
# Set P10 to P90  quantile
P10 = 0.1
P20=0.2
P30=0.3
P40=0.4
P50=0.5
P60=0.6
P70=0.7
P80=0.8
P90 = 0.9
# Each model has to be separate
P10_model = GradientBoostingRegressor(loss="quantile",                   
                                        alpha=P10, n_estimators=N_ESTIMATORS, max_depth=MAX_DEPTH)
P20_model = GradientBoostingRegressor(loss="quantile",                   
                                        alpha=P20, n_estimators=N_ESTIMATORS, max_depth=MAX_DEPTH)
P30_model = GradientBoostingRegressor(loss="quantile",                   
                                        alpha=P30, n_estimators=N_ESTIMATORS, max_depth=MAX_DEPTH)
P40_model = GradientBoostingRegressor(loss="quantile",                   
                                        alpha=P40, n_estimators=N_ESTIMATORS, max_depth=MAX_DEPTH)
P50_model = GradientBoostingRegressor(loss="quantile",                   
                                        alpha=P50, n_estimators=N_ESTIMATORS, max_depth=MAX_DEPTH)
P60_model = GradientBoostingRegressor(loss="quantile",                   
                                        alpha=P60, n_estimators=N_ESTIMATORS, max_depth=MAX_DEPTH)
P70_model = GradientBoostingRegressor(loss="quantile",                   
                                        alpha=P70, n_estimators=N_ESTIMATORS, max_depth=MAX_DEPTH)
P80_model = GradientBoostingRegressor(loss="quantile",                   
                                        alpha=P80, n_estimators=N_ESTIMATORS, max_depth=MAX_DEPTH)
P90_model = GradientBoostingRegressor(loss="quantile",                   
                                        alpha=P90, n_estimators=N_ESTIMATORS, max_depth=MAX_DEPTH)


### 5- Create training and testing data set

In [318]:
from sklearn.model_selection import train_test_split
y=df['Load']
df=df.drop(['Load'],axis=1)
df=df.drop(['DATE'],axis=1)
split_point = len(df) - (365*24)
X_train, X_test = df[0:split_point], df[split_point:]
y_train, y_test = y[0:split_point], y[split_point:]

In [319]:
P10_model.fit(X_train, y_train)
P20_model.fit(X_train, y_train)
P30_model.fit(X_train, y_train)
P40_model.fit(X_train, y_train)
P50_model.fit(X_train, y_train)
P60_model.fit(X_train, y_train)
P70_model.fit(X_train, y_train)
P80_model.fit(X_train, y_train)
P90_model.fit(X_train, y_train)
# Record actual values on test set
predictions = pd.DataFrame(y_test)

In [320]:
# Predict
predictions['P10_model'] = P10_model.predict(X_test)
predictions['P20_model'] = P20_model.predict(X_test)
predictions['P30_model'] = P30_model.predict(X_test)
predictions['P40_model'] = P40_model.predict(X_test)
predictions['P50_model'] = P50_model.predict(X_test)
predictions['P60_model'] = P60_model.predict(X_test)
predictions['P70_model'] = P70_model.predict(X_test)
predictions['P80_model'] = P80_model.predict(X_test)
predictions['P90_model'] = P90_model.predict(X_test)


### 6- Accuracy Calculation

In [321]:
MAPE_10=np.mean(np.abs((predictions["Load"] - predictions["P10_model"]) / predictions["Load"])) * 100
MAPE_50=np.mean(np.abs((predictions["Load"] - predictions["P50_model"]) / predictions["Load"])) * 100
MAPE_90=np.mean(np.abs((predictions["Load"] - predictions["P90_model"]) / predictions["Load"])) * 100
print("MAPE for P10 model: ",MAPE_10)
print("MAPE for P50 model: ",MAPE_50)
print("MAPE for P90 model: ",MAPE_90)

MAPE for P10 model:  11.214523124500968
MAPE for P50 model:  6.239319355249965
MAPE for P90 model:  6.493246038277823


### 7- Making the Forecast

In [322]:
df_prediction_data = pd.read_excel("Fcst 2007.xlsx")
df_prediction_data = df_prediction_data.dropna(axis = 0) # Remove rows with NaN data 
df_prediction_data = df_prediction_data.reset_index(drop = True) # Replace the previous index with new index
df_prediction_data["DATE"] = pd.to_datetime(df_prediction_data["DATE"], format = "%Y-%m-%d") # Specify the format of entry date
df_prediction_data["Year"] = df_prediction_data["DATE"].dt.year
df_prediction_data["Month"] = df_prediction_data["DATE"].dt.month
df_prediction_data["Day"] = df_prediction_data["DATE"].dt.day
df_prediction_data["Weekday"] = df_prediction_data["DATE"].dt.weekday + 1 # To adjust to 1 to 7 instead of 0 to 6
df_prediction_data["Week"] = df_prediction_data["Day"].apply(lambda x: (x - 1) // 7 + 1)
df_prediction_data["Weekend"] = df_prediction_data["Weekday"].apply(lambda x: 1 if x >= 6 else 0) # Apply function to the column (default: axis=0)
df_prediction_data

,DATE,Hour,Temperature,Year,Month,Day,Weekday,Week,Weekend
0,2007-01-01,1,55.992,2007,1,1,1,1,0
1,2007-01-01,2,55.448,2007,1,1,1,1,0
2,2007-01-01,3,55.008,2007,1,1,1,1,0
3,2007-01-01,4,54.672,2007,1,1,1,1,0
4,2007-01-01,5,54.304,2007,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...
8755,2007-12-31,20,64.720,2007,12,31,1,5,0
8756,2007-12-31,21,63.520,2007,12,31,1,5,0
8757,2007-12-31,22,62.864,2007,12,31,1,5,0
8758,2007-12-31,23,62.256,2007,12,31,1,5,0


In [323]:
df_prediction_data["Holiday"] = 0
df_prediction_data["Holiday"] = df_prediction_data["DATE"].apply(lambda x: 1 if (datetime.datetime.timestamp(x) in last_mon_may) else 0)
df_prediction_data["Holiday"].loc[(df_prediction_data["Month"] == 1) & (df_prediction_data["Day"] == 1)] = 1 # Remember () for condition
df_prediction_data["Holiday"].loc[(df_prediction_data["Month"] == 12) & (df_prediction_data["Day"] == 25)] = 1
df_prediction_data["Holiday"].loc[(df_prediction_data["Month"] == 11) & (df_prediction_data["Day"] == 11)] = 1
df_prediction_data["Holiday"].loc[(df_prediction_data["Month"] == 7) & (df_prediction_data["Day"] == 4)] = 1
df_prediction_data["Holiday"].loc[(df_prediction_data["Month"] == 1) & (df_prediction_data["Week"] == 3) & (df_prediction_data["Weekday"] == 1)] = 1
df_prediction_data["Holiday"].loc[(df_prediction_data["Month"] == 2) & (df_prediction_data["Week"] == 3) & (df_prediction_data["Weekday"] == 1)] = 1 
df_prediction_data["Holiday"].loc[(df_prediction_data["Month"] == 11) & (df_prediction_data["Week"] == 4) & (df_prediction_data["Weekday"] == 4)] = 1
df_prediction_data["Holiday"].loc[(df_prediction_data["Month"] == 10) & (df_prediction_data["Week"] == 2) & (df_prediction_data["Weekday"] == 1)] = 1
df_prediction_data["Holiday"].loc[(df_prediction_data["Month"] == 9) & (df_prediction_data["Week"] == 1) & (df_prediction_data["Weekday"] == 1)] = 1

print ("Total number for holidays: {} \n".format(np.sum(df_prediction_data["Holiday"])))

Total number for holidays: 216 



In [324]:
df_prediction_data = df_prediction_data[["DATE", "Hour","Temperature","Year","Month","Day","Weekday","Week","Weekend","Holiday"]]

In [325]:
df_prediction_data_backup=df_prediction_data.copy()

In [326]:
df_prediction_data=df_prediction_data.drop(['DATE'],axis=1)

In [327]:
predictions_data = pd.DataFrame(y_test)

In [328]:
# Predict
predictions_data['P10_model'] = P10_model.predict(df_prediction_data)
predictions_data['P20_model'] = P20_model.predict(df_prediction_data)
predictions_data['P30_model'] = P30_model.predict(df_prediction_data)
predictions_data['P40_model'] = P40_model.predict(df_prediction_data)
predictions_data['P50_model'] = P50_model.predict(df_prediction_data)
predictions_data['P60_model'] = P60_model.predict(df_prediction_data)
predictions_data['P70_model'] = P70_model.predict(df_prediction_data)
predictions_data['P80_model'] = P80_model.predict(df_prediction_data)
predictions_data['P90_model'] = P90_model.predict(df_prediction_data)

### 8- Converting the Forecasts into .csv file format

In [329]:
predictions_data['P10_model'].to_csv('P10_model.csv',header=True)
predictions_data['P20_model'].to_csv('P20_model.csv',header=True)
predictions_data['P30_model'].to_csv('P30_model.csv',header=True)
predictions_data['P40_model'].to_csv('P40_model.csv',header=True)
predictions_data['P50_model'].to_csv('P50_model.csv',header=True)
predictions_data['P60_model'].to_csv('P60_model.csv',header=True)
predictions_data['P70_model'].to_csv('P70_model.csv',header=True)
predictions_data['P80_model'].to_csv('P80_model.csv',header=True)
predictions_data['P90_model'].to_csv('P90_model.csv',header=True)